# Alur Preprocessing 1 (Text feature Extraction - POS-Tagging)

Alur dalam preprocessing ini yaitu dengan mengekstraksi kata-kata dengan beberapa metode. Alurnya adalah sebagai berikut

1. Load Data
2. POS-Tagging
4. Vectorize
5. Analyze

In [1]:
import matplotlib.pyplot as plt
import os, re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from datetime import datetime as dt
import cPickle as pickle
import re #Regex
%matplotlib inline

Tidak bisa digabung dengan STEM dan Stopword Removal, karena POS-tagging akan maksimal hasilnya jika kalimat terstruktur, jika sudah di stem atau stopwordnya dihapus, maka strukturnya tidak akan bagus, sehingga dipakai yang belum di strip

In [2]:
#just load it from the pickle dump
DATA_JUDUL = pickle.load( open( "v1.4\DATA_JUDULStriped-v2.p", "rb" ) )
DATA_SHORT = pickle.load( open( "v1.4\DATA_SHORTStriped-v2.p", "rb" ) )
DATA_LONG = pickle.load( open( "v1.4\DATA_LONGStriped-v2.p", "rb" ) )
DATA_TARGET = pickle.load( open( "v1.4\DATA_TARGET-v2.p", "rb" ) )

In [3]:
%time postagger = pickle.load(open( "POSTAGGER.p", "rb" ))

Wall time: 4.8 s


In [4]:
import string

def onlyAZ(teks):
    return re.sub(r'[^a-zA-Z]',' ', teks)
    
def onlyNVFromSentence(teks):
    splitted = postagger.tag(onlyAZ(teks).split())
    nouns = [word for word,pos in splitted \
        if (pos == 'NN' or pos == 'NNP' or 
            pos == 'NNS' or pos == 'NNPS'or 
            pos == 'VB')]
    nounsSentence = string.join(nouns)
    return nounsSentence

def onlyNVFromParagraph(par):
    splittedPar = par.split('.')
    nounsPar = string.join([onlyNVFromSentence(i) for i in splittedPar])
    return  nounsPar.strip() 

In [5]:
#Time saver strategy
%time DATA_JUDULPos = DATA_JUDUL.map(lambda x: onlyNVFromParagraph(x))
%time DATA_SHORTPos = DATA_SHORT.map(lambda x: onlyNVFromParagraph(x))
%time DATA_LONGPos  = DATA_LONG.map(lambda x: onlyNVFromParagraph(x))

Wall time: 671 ms
Wall time: 1.47 s
Wall time: 21.3 s


In [6]:
print DATA_SHORT[98]
print DATA_SHORTPos[98]

permintaan kredit di enam sektor ekonomi melambat.
permintaan kredit sektor ekonomi melambat


## Vektorisasi

##### Contoh Vektorisasi

In [8]:
import pandas as pd

#Stopwords data source : https://www.illc.uva.nl/Research/Publications/Reports/MoL-2003-02.text.pdf
stopword_html = open("id.stopwords.01.01.2016.txt",'r').read()
stopwords     = stopword_html.split("\n")

ff = ['Ini adalah KALIMAT CONTOH!!',
      'KaLiMat contoh ada 5800',
      'kalimat yang dengan baik,kalimaT yang buruk,dan kalimat yang sedang 2016']
dummy  = ff
countv = TfidfVectorizer(stop_words=stopwords,decode_error='ignore')
dummy  = countv.fit_transform(dummy)

In [9]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(dummy.toarray(), columns=countv.get_feature_names())

,2016,5800,buruk,contoh,kalimat
0,0.000000,0.000000,0.000000,0.789807,0.613356
1,0.000000,0.720333,0.000000,0.547832,0.425441
2,0.441105,0.000000,0.441105,0.000000,0.781571


Vektorisasi sudah termasuk menghilangkan karakter-karakter selain alphanumeric dan mengubah seluruh huruf menjadi huruf kecil. Kata 'ini', 'baik' , 'dengan' , dan 'sedang' hilang karena masuk dalam list stopwords

## Looking For Insight

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

def tableTop10(a,b,c):    
    def topKWords(docs,k):
        dummy  = docs.tolist()
        countv = CountVectorizer(stop_words=stopwords,decode_error='ignore')
        dummy  = countv.fit_transform(dummy)
        freqs = [(word, dummy.getcol(idx).sum()) for word, idx in countv.vocabulary_.items()]
        #sort from largest to smallest
        return [name for name,freq in sorted(freqs, key = lambda x: -x[1])[0:k]]
    
    top_word = pd.DataFrame([topKWords(a,10),
                             topKWords(b,10),
                             topKWords(c,10)]).transpose()
    return top_word

In [11]:
top_wordBerita1 = tableTop10(DATA_JUDUL,DATA_SHORT,DATA_LONG)
top_wordBerita1.columns = ['Judul','Short','Long']
top_wordBerita2 = tableTop10(DATA_JUDULPos,DATA_SHORTPos,DATA_LONGPos)
top_wordBerita2.columns = ['JudulPOSTag','ShortPOSTag','LongPOSTag']

In [12]:
top_wordBerita = [top_wordBerita1,top_wordBerita2]
top_wordBerita = pd.concat(top_wordBerita,axis=1)
print '10 Kata paling tinggi Frekuensinya Pada teks Sebelum preprocessing'
top_wordBerita

10 Kata paling tinggi Frekuensinya Pada teks Sebelum preprocessing


,Judul,Short,Long,JudulPOSTag,ShortPOSTag,LongPOSTag
0,com,indonesia,indonesia,rp,indonesia,indonesia
1,jpnn,pt,persen,com,pt,rp
2,indonesia,jakarta,2016,jpnn,jakarta,jakarta
3,harga,pemerintah,jakarta,indonesia,rp,harga
4,jokowi,harga,harga,harga,pemerintah,pemerintah
5,rumah,2016,pemerintah,jokowi,harga,negara
6,saham,persen,negara,rumah,bank,pt
7,ekonomi,bank,pt,saham,saham,rumah
8,pemerintah,saham,rumah,ekonomi,negara,perusahaan
9,triliun,negara,perusahaan,pemerintah,rumah,pasar


Terlihat bahwa perbedaan hasilnya cukup baik. Hal ini terkait dengan hilangnya beberapa kata yang tidak mengandung makna yang signifikan terhadap informasi yang dikandungnya. Contoh beberapa kata tersebut adalah 'triliun' dan 'persen'. Kata 'triliun' jika didalam kalimat kutipan maka tidak dihilangkan'

In [13]:
onlyNVFromParagraph('"dengan repatriasi yang bisa mendatangkan dana rp 500 triliun, \
akan berpengaruh menggairahkan pasar modal, pasar uang dan perbankan. bahkan, bisa menggerakkan sektor riil (lapangan pekerjaan),"')

'repatriasi mendatangkan dana rp berpengaruh menggairahkan pasar modal pasar uang perbankan menggerakkan sektor riil lapangan pekerjaan'

In [7]:
pickle.dump( DATA_JUDULPos, open( "v1.6\DATA_JUDULpostag-v2.p", "wb" ) )
pickle.dump( DATA_SHORTPos, open( "v1.6\DATA_SHORTpostag-v2.p", "wb" ) )
pickle.dump( DATA_LONGPos, open( "v1.6\DATA_LONGpostag-v2.p", "wb" ) )
pickle.dump( DATA_TARGET, open( "v1.6\DATA_TARGET-v2.p", "wb" ) )